In [131]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [132]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import time
import datetime as dt
import os

In [133]:
#package_dir = os.path.dirname(os.path.abspath('FICHIER DU 09 BOOST SAVE'))
#LeChemin = '/'+package_dir+'/20191211 BACKTEST/'
LeChemin = 'DL_2019-12-06/'
start = '2015-12-06'
end = '2020-02-07'
list_dir = os.listdir('DL_2019-12-06/.')
NEW_LIST = []
for pikunichihouahoua in range(0,len(str(list_dir).split(','))):
    if str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1])
NEW_LIST = sorted(set(NEW_LIST))

In [134]:
___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5780(c)')
print(Fore.BLUE,'°Go!em 73°')
print('ULTIMATE BACKTEST v0.5',Style.RESET_ALL)

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5780(c)
 °Go!em 73°
ULTIMATE BACKTEST v0.5 


In [135]:
def scrap_data(ticker,start,end):
    df = web.DataReader(ticker,'yahoo',start,end)
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)    
    return(df)

def load_boot(df,model):
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    return(df)


def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        rsi70_list.append(70 - rsi[i])
        rsi30_list.append(rsi[i] - 30)
        #except:
         #   rsi70_list.append(0)
          #  rs30_list.append(0)
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    
    df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]):
        if df.iloc[i]['%Futur'] > 0.5 :#or df.iloc[i]['%Futur2'] > 0.1:
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -0.5: #or df.iloc[i]['%Futur2'] < -0.1:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)

    df['target_up'] = up  # target_up # abs(np.array(valley))#target_up
    df['target_down'] = down # target_down # abs(np.array(peak))#target_down
    df = df[['High','Low','Open','Volume','Close','Varop_Spy','Varhl_spy','RSI','70 - RSI','RSI - 30',\
             'BBD_Delta_Up','delta5_8','delta8_10','delta10_12','delta12_15','delta15_30','delta30_35',\
             'delta35_40','delta40_45','delta45_50','Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper',\
             'lower','var_bollup_kchigh','var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon','Close.S',\
             'Close.S2','%Futur','%Futur2','target_up','target_down']]
    #df = df.dropna()
    return(df)

def deep_learning(df):
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    #####################X = X.iloc[:,1:-4]
    X = X.iloc[:,:-4]
    X.astype(np.float64)
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 

    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:].strftime('%Y-%m-%d')
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,scaler)

In [ ]:
###################################################
########### GENERATION DE BASE ####################
###################################################

print(Fore.YELLOW,'ON GENERE LaBase',Style.RESET_ALL)
tmps55=time.time()
LaBase = pd.DataFrame()
df = pd.DataFrame()
BADTICKER = ['AMCR']
for loop,ticker in enumerate(NEW_LIST):
    try:
        df = web.DataReader(ticker,'yahoo',start,end)
        df['Date'] = df.index
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    except:
        BADTICKER.append(ticker)
        continue
    df['Symbol'] = ticker
    df = df[['Symbol','High','Low','Open','Close','Volume','Adj Close']]
    LaBase = pd.concat((LaBase,df),ignore_index=False)
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for completing LaBase Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

print('Il y a ',len(BADTICKER),' mauvais tickers')
print('Les voici')
print(BADTICKER)

#LaBase[LaBase.duplicated()].shape

print('Base générée : ',LaBase.shape)
print('Il y a ',LaBase[LaBase.duplicated()].shape,' Duplicats')
print('Hors AMCR, voici les duplicats') 
print(LaBase[(LaBase.duplicated()==True)&(LaBase['Symbol'] != 'AMCR')])

###########################################################
############## SI BESOIN DE RETIRER LA DATE DU JOUR #######
##
#  LaBase = LaBase[LaBase.index != '2019-09-18 00:00:00'] #
##
#
# LaBase.index = pd.to_datetime(LaBase['Date'],format='%Y-%m-%d')
###########################################################
try:
    os.remove('LaBase_17.csv')
except:
    pass
LaBase.to_csv('LaBase_17.csv') 

In [136]:
import talib
from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
yaml.warnings({'YAMLLoadWarning': False})
warnings.filterwarnings("ignore")
import pickle

In [137]:
LaBase = pd.read_csv('LaBase_adj.csv')
try:
    LaBase = LaBase.drop(['Unnamed: 0'],axis=1)
except :
    pass
LaBase.index = pd.to_datetime(LaBase['Date'],format='%Y-%m-%d')
try:
    LaBase = LaBase.drop(['Date'],axis=1)
    #LaBase = LaBase.drop(['Close'],axis=1)
    LaBase['Close'] = LaBase['Adj Close']
except:
    pass
try:
    LaBase = LaBase.drop(['Adj Close'],axis=1)
except:
    pass
LaBase,end

(           Symbol        High         Low        Open     Volume       Close
 Date                                                                        
 2015-12-07      A   41.099998   40.540001   40.919998  2004500.0   39.091465
 2015-12-07    AWK   58.410000   57.590000   57.750000   783000.0   53.948914
 2015-12-07   CINF   61.560001   61.060001   61.299999   683500.0   54.486759
 2015-12-07     CL   66.559998   66.089996   66.389999  3895600.0   60.412056
 2015-12-07    IEX   78.620003   77.320000   78.400002   302800.0   73.439034
 ...           ...         ...         ...         ...        ...         ...
 2020-02-07    XYL   84.910004   83.129997   83.360001  2496800.0   84.199997
 2020-02-07    YUM  102.620003  100.750000  101.760002  2396800.0  102.040001
 2020-02-07    ZBH  158.729996  156.119995  158.279999   889300.0  156.389999
 2020-02-07   ZION   46.840000   45.950001   46.840000  2611800.0   46.230000
 2020-02-07    ZTS  139.160004  138.029999  139.050003  1780800.

In [138]:
long = len(NEW_LIST)


BasePrep = pd.read_csv('BasePrep.csv')
try:
    BasePrep = BasePrep.drop(['Unnamed: 0'],axis=1)
except :
    pass
BasePrep.index = pd.to_datetime(BasePrep['Date'],format='%Y-%m-%d')
try:
    BasePrep = BasePrep.drop(['Date'],axis=1)
    #LaBase = LaBase.drop(['Close'],axis=1)
    BasePrep['Close'] = BasePrep['Adj Close']
except:
    pass
try:
    BasePrep = BasePrep.drop(['Adj Close'],axis=1)
except:
    pass
BasePrep = BasePrep[['Symbol','High','Low','Open','Volume','Close',\
                         'Varop_Spy','Varhl_spy','RSI','70 - RSI','RSI - 30',\
                         'BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
                         'delta12_15','delta15_30','delta30_35','delta35_40',\
                         'delta40_45','delta45_50','Stoc_Slowk','Stoc_Slowd',\
                         'KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
                         'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon',\
                         'Close.S','Close.S2','%Futur','%Futur2','target_up','target_down']]

LaBase = LaBase[(LaBase.index>start)&(LaBase.index<=end)]

BasePrep = BasePrep[(BasePrep.index>start)&(BasePrep.index<=end)]


print('Dernière date de BasePrep :',str(BasePrep.index[-1])[:10])

delta = round(LaBase.shape[0])
bloc1 =  round(0.75 * LaBase[LaBase['Symbol']=='A'].shape[0]) #round(delta*0.80)
bloc2 = delta - bloc1
print('Bloc1 = ',bloc1)

signals = pd.DataFrame(columns = ['Date','Ticker','Open Long','Close Long','Open Short','Close Short','Close_position_price','Buy/Sell Price',\
                                  'Quantity','Actual','PNL','Cumulative PNL','Latent PNL','Duration '])


backtest = pd.DataFrame(columns = ['Symb.','Name','Sector','Period','Begin.','End',\
                          'Pnl $','Pnl %','Gross Profit','Gross Loss','# Winners','# Loosers',\
                          '% Winners','MaxDD $','MaxDD %','Aver. Win $',\
                          'Aver. Loos $','Aver. Duration',\
                          'Total Trades','% Long','% Short','Profit Factor'])


print(Fore.GREEN,'Initiating Calculation',Style.RESET_ALL)

Resultats = pd.DataFrame()

GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE=[]
TICKER = []
DATE = []
PRICE = []
OPEN_S = []
CONFUP = []
CONFDWN = []
PERUP = []
PERDOWN = []

ERRORDATE = []
ERRORLONG = []

tmps55=time.time()
for loop,ticker in enumerate(NEW_LIST):
    
    long = len(NEW_LIST)

    if loop == 1:
            print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
    if loop == round(1 * long / 9):
        print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
    if loop == round(2 * long / 9):
        print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
    if loop == round(3 * long / 9):
        print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
    if loop == round(4 * long / 9):
        print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
    if loop == round(5 * long / 9):
        print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
    if loop == round(6 * long / 9):
        print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
    if loop == round(7 * long / 9):
        print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
    if loop == round(8 * long / 9):
        print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',Style.RESET_ALL,end='',flush=True)
    
    
    LeChemin = 'DL_2019-12-06/'
    yamlup = (LeChemin+'Save_'+ticker+'_up.yaml')
    yamldown = (LeChemin+'Save_'+ticker+'_down.yaml')
    modelup = (LeChemin+'Save_'+ticker+'_up.h5')
    modeldown = (LeChemin+'Save_'+ticker+'_down.h5')

    yaml_file_up = open(yamlup, 'r')
    yaml_file_down = open(yamldown, 'r')
    model_yaml_up = yaml_file_up.read()
    model_yaml_down = yaml_file_down.read()
    yaml_file_up.close()
    yaml_file_down.close()
    model_up = model_from_yaml(model_yaml_up)
    model_down = model_from_yaml(model_yaml_down)
    
    # load weights into new model
    model_up.load_weights(modelup)
    model_down.load_weights(modeldown)
        
    df = BasePrep[BasePrep['Symbol']==ticker]
    if df.shape[0] < 70:
        ERRORLONG.append(ticker)
        continue
    elif str(df.index[-1])[:10] != end :
        ERRORDATE.append(ticker)
        continue
    
    df = df.drop(['Symbol'],axis=1)
                 
    df = df[['High','Low','Open','Volume','Close','Varop_Spy',\
             'Varhl_spy','RSI','70 - RSI','RSI - 30','BBD_Delta_Up',\
             'delta5_8','delta8_10','delta10_12','delta12_15','delta15_30',\
             'delta30_35','delta35_40','delta40_45','delta45_50',\
             'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper',\
             'lower','var_bollup_kchigh','var_bolllow_kclow','Aroon Up',\
             'Aroon Down','Delta Aroon','Close.S','Close.S2','%Futur','%Futur2','target_up','target_down']]
    
    df.to_csv('PrepData/prepdata_'+ticker+'_Live.csv')
    
    resultats,precision_up,precision_down,scaler = deep_learning(df)
    
    
    #if len(resultats) < 252:
     #   error_2.append(ticker)
      #  continue
    
    if (precision_up * 100) > 87 and (precision_down * 100) > 87:

        ####################
        ##### SIGNALS #####
        ###################

        filtre_up = 93
        filtre_down = 97
        filtre_up_close = 90
        filtre_down_close = 90

        for i in range(0,len(resultats)):

            price = resultats.iloc[i]['Actual']

            if resultats.iloc[i]['Confiance Down'] > filtre_down:
                doob = -1
                waab = 0
                

            elif resultats.iloc[i]['Confiance Down'] > filtre_down_close and resultats.iloc[i]['Confiance Down'] < filtre_down :
                waab = -1
                doob = 0
                

            elif resultats.iloc[i]['Confiance up'] > filtre_up:
                doob = 1
                waab = 0
                

            elif resultats.iloc[i]['Confiance up'] > filtre_up_close and resultats.iloc[i]['Confiance up'] < filtre_up:
                waab = 1
                doob = 0
                

            else :
                waab = 0
                doob = 0
                

            if doob == 1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(1)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue

            if doob == -1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(1)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
                
            if doob == 0 and waab == -1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
                
            if doob == 0 and waab == 1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
                
            if doob == 0 and waab == 0 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(999)
                RED.append(999)
                GREEN_CLOSE.append(999)
                RED_CLOSE.append(999)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                doob = 0
                waab = 0
                continue
                
                
    '''else:
        DATE.append(resultats.iloc[i]['Date'])
        TICKER.append(ticker)
        GREEN.append(0)
        RED.append(0)
        GREEN_CLOSE.append(0)
        RED_CLOSE.append(0)
        PRICE.append(price)
        OPEN_S.append(resultats.iloc[i]['Open.S'])
        CONFUP.append(resultats.iloc[i]['Confiance up'])
        CONFDWN.append(resultats.iloc[i]['Confiance Down'])'''
        

Resultats['Date'] = DATE
Resultats['Ticker'] = TICKER
Resultats['Conf Up'] = CONFUP
Resultats['Conf Dwn'] = CONFDWN
Resultats['Green'] = GREEN
Resultats['Red'] = RED
Resultats['Green_Close'] = GREEN_CLOSE
Resultats['Red_Close'] = RED_CLOSE
Resultats['Price'] = PRICE
Resultats['Open.S'] = OPEN_S
Resultats['% up'] = PERUP
Resultats['% down'] = PERDOWN

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing Signal Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
Resultats = Resultats[(Resultats['Green_Close']==1)|(Resultats['Red_Close']==1)]
Resultats.to_csv('Resultats_live.csv')


Dernière date de BasePrep : 2020-02-07
Bloc1 =  787
 Initiating Calculation 
   |==================  O  ==|  
 Time for completing Signal Generation = 191.310000 seconds 
 


In [144]:
len(ERRORDATE),len(ERRORLONG)

(5, 46)

In [154]:
loop,ticker,sum(GREEN),sum(GREEN_CLOSE),sum(RED),sum(RED_CLOSE),len(GREEN),len(set(TICKER)),DATE[-1]

(342, 'ZION', 4655340, 4655340, 4655340, 4655340, 4660, 22, '2020-02-07')

In [146]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import sys
import codecs
%matplotlib inline
plt.style.use('seaborn')
orig_stdout = sys.stdout

In [147]:
########################
####### BACKTEST########
########################
print('Début du backtesting...')
tmps55=time.time()
LeChemin=''
resultats = Resultats
resultats['Date'] = pd.to_datetime(resultats['Date'],format='%Y-%m-%d')
tmps55=time.time()
error_3 = []
for ticker in NEW_LIST:
    if ticker in ERRORLONG or ticker in ERRORDATE:
        continue
    pos_long = 0
    pos_short = 0
    pactol = 1000000
    mise = 30000
    price_buy = 0
    price_sell = 0
    x = 0
    winners = []
    loosers = []
    nb_win = 0
    nb_los = 0
    mini_pnl = 0
    pos_duration = []
    average_duration = [] # pd.Timedelta(resultats.iloc[0,0] - resultats.iloc[0,0])
    trades = []
    eq_curx = []
    eq_cury = []
    maxdd = []
    nbS = 0
    nbL = 0
    
    
    f = codecs.open('BT/backtest de '+str(ticker)+'.txt', 'w','utf-8')
    f.write(u'\ufeff')
    sys.stdout = f
    print('TICKER : ',ticker)
    resultats = Resultats[Resultats['Ticker'] == ticker]
    if resultats.shape[0] < 1:
        error_3.append(ticker)
        continue
    for i in range(0,len(resultats)):
        if pos_long == 0 and pos_short == 0 and resultats.iloc[i]['Green'] == 1 :
            pos_long = 1
            nbL+=1
            x = round((mise)/resultats.iloc[i]['Open.S'])
            price_buy = round(resultats.iloc[i]['Open.S'],2)
            time_enter = resultats.iloc[i]['Date']

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 1,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,'Close_position_price':0,\
                               'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),\
                                       'Latent PNL':0,'Duration ':0}],ignore_index=True)

            print('\n Le ',resultats.iloc[i]['Date'].strftime('%Y-%m-%d'),', achat de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',round(pactol,2))
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],1))
            continue

        elif pos_long == 1 and resultats.iloc[i]['Green_Close'] == 0 :

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,'Close_position_price':0,\
                               'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : 0,\
                                       'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                       'Latent PNL':round((resultats.iloc[i]['Price'] - price_buy)*x,2),'Duration ':0}],ignore_index=True)
            try:
                maxdd.append(round((resultats.iloc[i]['Open.S'] - price_buy)*x,2))
            except:
                print('Null val of maxdd')
                maxdd.append(0)
            print('\n Le ',str(resultats.iloc[i]['Date']))
            print('Nouveau pactol : ',round(pactol,2))
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : ',maxdd[-1])
            continue

        elif pos_long == 1 and resultats.iloc[i]['Green_Close'] == 1 :
            old_pactol = round(pactol,2)
            eq_curx.append(resultats.iloc[i]['Date'])
            eq_cury.append(old_pactol)
            pos_long = 0
            price_sell = round(resultats.iloc[i]['Open.S'],2)
            mini_pnl = round(x*(price_sell-price_buy),2)
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],2))

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,'Close_position_price':price_sell,\
                               'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : mini_pnl,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0,\
                                       'Duration ':str(pd.to_timedelta((resultats.iloc[i]['Date'] - time_enter),unit='d'))[:-8]}],ignore_index=True)

            pactol = round((pactol + mini_pnl),2)
            pos_duration.append(resultats.iloc[i]['Date'] - time_enter)
            print('\n Le ',str(resultats.iloc[i]['Date']),', vente pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)
            print('pnl:', mini_pnl)
            print('prix de vente',price_sell)
            print('prix d achat',price_buy)
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            print('Duration : ',pos_duration[-1])
            

            if mini_pnl > 0:
                winners.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                loosers.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1
            continue


        elif pos_short == 0 and pos_long == 0 and resultats.iloc[i]['Red']==1:
            pos_short = 1
            nbS+=1
            x = round((mise)/resultats.iloc[i]['Open.S'])
            price_sell = round(resultats.iloc[i]['Open.S'],2)
            time_enter = resultats.iloc[i]['Date']

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 1, 'Close Short' : 0,'Close_position_price':0,\
                               'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0,'Duration ':0}],ignore_index=True)

            print('\n Le ',str(resultats.iloc[i]['Date']),', vente de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-1))
            continue


        elif pos_short == 1 and resultats.iloc[i]['Red_Close'] == 0 :

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,'Close_position_price':0,\
                               'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),\
                                       'Latent PNL':-round((resultats.iloc[i]['Price'] - price_sell)*x,2),'Duration ':0}],ignore_index=True)
            try:
                maxdd.append(-round((resultats.iloc[i]['Open.S'] - price_sell)*x,2))
            except:
                print('Null val of maxdd')
                maxdd.append(0)
                print('\n Le ',str(resultats.iloc[i]['Date']))
                print('Nouveau pactol : ',round(pactol,2))
                print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
                print('Le Latent PNL est de : ',maxdd[-1])
            continue

        elif pos_short == 1 and resultats.iloc[i]['Red_Close'] == 1 :
            old_pactol = round(pactol,2)
            eq_curx.append(resultats.iloc[i]['Date'])
            eq_cury.append(old_pactol)
            pos_short = 0
            price_buy = round(resultats.iloc[i]['Open.S'],2)
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-2))
            mini_pnl = round(x*(price_sell-price_buy),2)

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,'Close_position_price':price_buy,\
                               'Buy/Sell Price' : price_sell,'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : mini_pnl,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0,\
                                       'Duration ':str(pd.to_timedelta((resultats.iloc[i]['Date'] - time_enter),unit='d'))[:-8]}],ignore_index=True)

            pactol = round((pactol + mini_pnl),2)
            print('\n Le ',str(resultats.iloc[i]['Date']),', achat pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)

            print('prix de vente',price_sell)
            print('prix d achat',price_buy)

            print('pnl:', mini_pnl)
            pos_duration.append(resultats.iloc[i]['Date'] - time_enter)
            
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            print('Duration : ',pos_duration[-1])
            
            if mini_pnl > 0:
                winners.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                loosers.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1
            continue
            
        else:

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,'Close_position_price':0,\
                           'Buy/Sell Price' : 0, 'Quantity' : 0,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0,'Duration ':0}],ignore_index=True)

    if pos_long == 1:
        pos_long = 0
        print('Cutting unresolved position')
        print(' Le price buy était de : ',price_buy)
        x = round((mise/resultats.iloc[i]['Open.S']))
        print('x = ',x)
        price_sell = round(resultats.iloc[i]['Open.S'],2)
        print('Et le price sell : ',price_sell)
        mini_pnl = round(x*(price_sell-price_buy),2)

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,'Close_position_price':price_sell,\
                               'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : mini_pnl,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0,\
                                   'Duration ':str(pd.to_timedelta((resultats.iloc[i]['Date'] - time_enter),unit='d'))[:-8]}],ignore_index=True)

        pactol = round((pactol + mini_pnl),2)
        print('pnl : ',mini_pnl)
        if mini_pnl > 0:
            winners.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            loosers.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1


    if pos_short == 1:
        pos_short = 0
        print('Cutting uresolved position')
        print(' Le price sell était de : ',price_sell)
        x = round((mise)/resultats.iloc[i]['Open.S'])
        print('x = ',x)
        price_buy = round(resultats.iloc[i]['Open.S'],2)
        print('Et le price_buy : ',price_buy)
        mini_pnl = round(x*(price_sell-price_buy),2)

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,'Close_position_price':price_buy,\
                               'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Open.S'],2), 'PNL' : mini_pnl,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0,\
                                   'Duration ':str(pd.to_timedelta((resultats.iloc[i]['Date'] - time_enter),unit='d'))[:-8]}],ignore_index=True)

        pactol = round((pactol + mini_pnl),2)
        print('pnl : ',mini_pnl)

        if mini_pnl > 0:
            winners.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            loosers.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1


    pnl = round(pactol - (1000000),2)
    print('\n Begining of BackTest :',resultats.iloc[0]['Date'])
    print('Instrument :',ticker)
    print('Face value per trade : $',mise)
    print('End of BackTest :',resultats.iloc[-1]['Date'])
    duration = pd.to_timedelta((resultats.iloc[-1]['Date'] - resultats.iloc[0]['Date']),unit='d')
    print("BackTest's period :",duration)
    print('Pnl :',round(pnl,2),'$')
    print('% Pnl : ',round(pnl/(mise)*100,2))
    if pnl == 0:
        continue
    print('Total winners :',round(sum(winners),2),'$')
    print('Total loosers :',round(sum(loosers),2),'$')
    print('Number of winners :',round(nb_win))
    print('Number of loosers :',round(nb_los))
    if nb_win == 0:
        nb_win += 0.00001
    if nb_los == 0:
        nb_los += 0.00001
    print('Maximum Drawdown : ',round(min(maxdd),2))
    print('% Max Drawdown : ',round(min(maxdd)/(mise)*100,2),'%')
    print('Nombre toatl de trades : ',round(nbL+nbS,2))
    print('% de trades Long : ',round((nbL*100)/(nbL+nbS),2))
    print('% de trades Short : ',round((nbS*100)/(nbL+nbS),2))
    try:
        print('% winners :', round((nb_win/(nb_win+nb_los) * 100),2),'%')
        print('Average winners :',(round(sum(winners)/(nb_win))))
    except:
        print('% winners : null')
    try:
        print('Average losers :',(round(sum(loosers)/(nb_los))))
    except:
        print('Average losers : Inf')
    try:
        average_duration = pd.Timedelta('0 days 00:00:00')
        for t in range(0,len(pos_duration)):
            average_duration = average_duration + pos_duration[t]
        average_duration = str(average_duration/(nb_los+nb_win))[:-7]
        print('Average Duration :', average_duration)
    except:
        print('No Average Duration available')
    try:
        print('Profit Factor : ',abs(round((sum(winners)/sum(loosers)),2)))
    except:
        print('Profit Factor : null')
        
    

    ########################
    ##### PLOT TRADES ######
    ########################

    #plt.figure(figsize=(26,8),dpi=300)
    #plt.title('Deep Learning '+ticker+' Trades')
    #plt.plot(resultats['Open.S'],color='blue',lw=0.9)
    #for i in range(0,len(trades)):
    #    if trades[i][2] == 1:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='green',marker='^',lw=2.5)
    #    elif trades[i][2] == -1:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='red',marker='^',lw=2.5)
    #    elif trades[i][2] == 2:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='g',marker='x',lw=2.5)
    #    elif trades[i][2] == -2:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='r',marker='x',lw=2.5)

    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='g',marker='^',label='Open Long')
    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='r',marker='^',label='Open Short')
    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='g',marker='x',label='Close Long')
    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='r',marker='x',label='Close Short')
    
    #plt.legend()
    #plt.savefig('backtest de '+str(ticker)+'_02.pdf')
    #plt.close()

    #########################
    ##### PLOT E-CURVE ######
    #########################
    plt.figure(figsize=(26,8),dpi=300)
    plt.title('Deep Learning '+ticker+' Equity Curve')
    plt.plot(eq_curx,eq_cury)
    plt.scatter(x=eq_curx,y=eq_cury,c='orange',marker='o',lw=2.5)
    plt.savefig('BT/backtest de '+str(ticker)+'_e_curve.pdf')
    plt.close()

    
    
    #print('\r Boucle ', loop, 'terminée...',end='')
    #print('Opération restantes :', len(constituents)-loop,'/',len(constituents),'\n')

    name = 'Name'
    sector = 'Sector'
    backtest = backtest.append([{'Symb.':ticker,'Name':name,'Sector':sector,'Period':average_duration,'Begin.':resultats.iloc[0,0],'End':resultats.iloc[-1,0],\
            'Pnl $':pnl,'Pnl %':round(pnl/(mise)*100,2),'Gross Profit':round(sum(winners),2),'Gross Loss':round(sum(loosers),2),'# Winners':nb_win,'# Loosers':nb_los,\
            '% Winners':round((nb_win/(nb_win+nb_los) * 100),2),'MaxDD $':round(min(maxdd),2),'MaxDD %':round(min(maxdd)/(mise)*100,2),'Aver. Win $':(round(sum(winners)/(nb_win+0.0001))),\
            'Aver. Loos $':(round(sum(loosers)/(nb_los+0.0001))),'Aver. Duration':average_duration,\
            'Total Trades':nbL+nbS,'% Long':round((nbL*100)/(nbL+nbS),2),'% Short':round((nbS*100)/(nbL+nbS),2), 'Profit Factor':abs(round((sum(winners)/sum(loosers)),2))}], ignore_index=True)


    #signals = signals.append([{ 'Date': list_signals[0], 'Ticker' : list_signals[8], 'Open Long' : list_signals[1],\
                               #'Close Long' : list_signals[2], 'Open Short' : list_signals[3], 'Close Short' : list_signals[4],\
                               #'Buy/Sell Price' : list_signals[5], 'Quantity' : list_signals[6], ' tPNL' : list_signals[7]}])

    #signals['Date'] = list_signals[0]
    #signals['Ticker'] = list_signals[8]
    #signals['Open Long'] = list_signals[1]
    #signals['Close Long'] = list_signals[2]
    #signals['Open Short'] = list_signals[3]
    #signals['Close Short'] = list_signals[4]
    #signals['Buy/Sell Price'] = list_signals[5]
    #signals['Quantity'] = list_signals[6]
    #signals['tPNL'] = list_signals[7]

    # symbol / name / sector / period of backtest / debut / fin / pnl / total winners / total loosers / Nb winners / nb loosers / 
    # / % winners / Max DD / Aver Win / Aver Loos / Average duration / Profit Factor
f.close()

sys.stdout = orig_stdout
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing the backtest = %f" %tmps22,'seconds \n',Style.RESET_ALL)
len(error_3)

Début du backtesting...
 
 Time for completing the backtest = 0.430000 seconds 
 


292

In [143]:
f.close()
sys.stdout = orig_stdout
#sys.stdout = sys.__stdout__

In [93]:
print(Fore.YELLOW,'Writing BACKTEST CSV AND SIGNALS.CSV',Style.RESET_ALL)


signals = signals[['Date','Ticker','Open Long','Close Long','Open Short','Close Short','Close_position_price','Buy/Sell Price','Quantity','Actual','PNL','Latent PNL','Duration ']]


backtest = backtest[['Symb.','Name','Sector','Period','Begin.','End',\
                          'Pnl $','Pnl %','Gross Profit','Gross Loss','# Winners','# Loosers',\
                          '% Winners','MaxDD $','MaxDD %','Aver. Win $',\
                          'Aver. Loos $','Aver. Duration',\
                          'Total Trades','% Long','% Short','Profit Factor']]



backtest.to_csv('BackTest_live.csv')
signals = signals.sort_values(by=['Date'])
#signals = signals.drop(['Cumulative PNL'],axis=1)
signals = signals.reset_index(drop=True)
#signals = signals[(signals['Close Long']==1)|(signals['Close Short']==1)]
signals.to_csv('SIGNALS_live.csv')

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing BackTest Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

print(Fore.CYAN,'BACKTEST CSV WRITED AND SIGNALS.CSV',Style.RESET_ALL)
    #except:
     #   print(Fore.RED,'Problème loop : ',loop,Style.RESET_ALL)
      #  error.append((loop,ticker))

        #continue

 Writing BACKTEST CSV AND SIGNALS.CSV 
 
 Time for completing BackTest Generation = 689.270000 seconds 
 
 BACKTEST CSV WRITED AND SIGNALS.CSV 


In [125]:
i,nb_los,nb_win,nbL,nbS,ticker,resultats.shape

(212, 0, 0, 0, 0, 'ZION', (0, 12))

In [126]:
len(error_3),error_3[0]

(292, 'A')

In [148]:
Resultats[Resultats['Date']>='2020-02-06']

,Date,Ticker,Conf Up,Conf Dwn,Green,Red,Green_Close,Red_Close,Price,Open.S,% up,% down


In [149]:
signals[(signals['Date']>='2014-02-06')&(signals['Ticker']=='CVX')]

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close_position_price,Buy/Sell Price,Quantity,Actual,PNL,Cumulative PNL,Latent PNL,Duration


In [150]:
Resultats[Resultats['Ticker']=='FFIV']

,Date,Ticker,Conf Up,Conf Dwn,Green,Red,Green_Close,Red_Close,Price,Open.S,% up,% down


In [130]:
str(pd.to_timedelta((resultats.iloc[i]['Date'] - time_enter),unit='d'))[:-8]

IndexError: single positional indexer is out-of-bounds

In [66]:
pos_duration

[Timedelta('1 days 00:00:00'),
 Timedelta('1 days 00:00:00'),
 Timedelta('1 days 00:00:00'),
 Timedelta('10 days 00:00:00'),
 Timedelta('1 days 00:00:00'),
 Timedelta('18 days 00:00:00'),
 Timedelta('90 days 00:00:00'),
 Timedelta('27 days 00:00:00'),
 Timedelta('41 days 00:00:00'),
 Timedelta('50 days 00:00:00')]

In [67]:
average_duration = pd.Timedelta('0 days 00:00:00')
for t in range(0,len(pos_duration)):
    average_duration = average_duration + pos_duration[t]
average_duration = str(average_duration/(nb_los+nb_win))[:7]
average_duration

'21 days'

In [151]:
BasePrep[(BasePrep['Symbol'] == 'MMM')&(BasePrep.index>='2015-04-05')].head(10)

,Symbol,High,Low,Open,Volume,Close,Varop_Spy,Varhl_spy,RSI,70 - RSI,...,var_bolllow_kclow,Aroon Up,Aroon Down,Delta Aroon,Close.S,Close.S2,%Futur,%Futur2,target_up,target_down
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-18,MMM,156.970001,155.970001,156.350006,2010200.0,141.609116,14.740891,1.000000,65.808852,4.191148,...,32.603324,88.888889,55.555556,33.333333,142.06240,142.06240,0.320091,0.320091,0,0
2016-02-19,MMM,157.089996,155.869995,157.000000,2121300.0,141.311050,15.688950,1.220001,64.813185,5.186815,...,32.157040,77.777778,44.444444,33.333333,141.71568,141.71568,0.286342,0.286342,0,0
2016-02-22,MMM,158.479996,156.449997,156.449997,2207600.0,142.440079,14.009918,2.029999,66.858588,3.141412,...,31.918463,100.000000,33.333333,66.666667,141.21736,141.21736,-0.858408,-0.858408,0,1
2016-02-23,MMM,157.690002,155.169998,157.630005,1922800.0,141.058151,16.571854,2.520004,62.100229,7.899771,...,31.560034,88.888889,22.222222,66.666667,142.01477,142.01477,0.678174,0.678174,1,0
2016-02-24,MMM,157.619995,153.639999,154.839996,1937800.0,142.006577,12.833420,3.979996,63.994207,6.005793,...,31.896529,77.777778,11.111111,66.666667,143.28458,143.28458,0.899958,0.899958,1,0
2016-02-25,MMM,159.000000,157.009995,157.539993,1718100.0,143.605270,13.934723,1.990005,66.988855,3.011145,...,33.602361,100.000000,0.000000,100.000000,142.87643,142.87643,-0.507527,-0.507527,0,1
2016-02-26,MMM,159.500000,158.080002,159.149994,1843700.0,142.954956,16.195038,1.419998,64.633932,5.366068,...,34.023113,100.000000,0.000000,100.000000,142.24382,142.24382,-0.497455,-0.497455,0,0
2016-02-29,MMM,159.119995,156.820007,158.009995,1993200.0,141.690445,16.319550,2.299988,60.202229,9.797771,...,34.107564,88.888889,66.666667,22.222222,143.92839,143.92839,1.579461,1.579461,1,0
2016-03-01,MMM,159.699997,157.610001,158.750000,2204100.0,144.147186,14.602814,2.089996,65.195317,4.804683,...,33.822354,100.000000,55.555556,44.444444,143.89656,143.89656,-0.173868,-0.173868,0,0


In [152]:
Resultats[(Resultats['Conf Up']<83)&(Resultats['Conf Dwn']<90)]

,Date,Ticker,Conf Up,Conf Dwn,Green,Red,Green_Close,Red_Close,Price,Open.S,% up,% down


In [153]:
Resultats

,Date,Ticker,Conf Up,Conf Dwn,Green,Red,Green_Close,Red_Close,Price,Open.S,% up,% down


In [43]:
end

'2020-02-07'

In [159]:
len(Resultats),len(Resultats[(Resultats['Green']==999)|(Resultats['Red']==999)])

(4660, 4660)